# Data Exploration - Team Gambinos

In [28]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import seaborn as sns
from sklearn.linear_model import LogisticRegression
from sklearn.metrics.pairwise import cosine_similarity
from nltk import word_tokenize, regexp_tokenize
from nltk.stem import WordNetLemmatizer 
from nltk.corpus import stopwords

In [2]:
complaints = pd.read_csv('../data/complaints.csv')

We decided as a team to adopt the following naming convention for consistency:

In [3]:
complaints.columns = [x.replace('Consumer ', '').lower().replace(' ', '_') for x in complaints.columns]
complaints['complaint_length'] = complaints['complaint_narrative'].apply(len)


In [4]:
complaints['issue'].value_counts()

Incorrect information on your report    229305
Attempts to collect debt not owed        73163
Communication tactics                    21243
Struggling to pay mortgage               17374
Fraud or scam                            12347
Name: issue, dtype: int64

In [5]:
complaints_dict = {'Attempts to collect debt not owed': 1, 
                 'Communication tactics': 2, 
                 'Fraud or scam': 3, 
                 'Incorrect information on your report': 4,
                 'Struggling to pay mortgage': 5}

In [6]:
# to replace the text in issue with target variables
complaints = complaints.replace({'issue': complaints_dict})

In [7]:
complaints['complaint_narrative']

0         My name is XXXX XXXX this complaint is not mad...
1         I searched on XXXX for XXXXXXXX XXXX  and was ...
2         I have a particular account that is stating th...
3         I have not supplied proof under the doctrine o...
4         Hello i'm writing regarding account on my cred...
                                ...                        
353427           Collections account I have no knowledge of
353428    Dear CFPB Team, The reason for my complaint is...
353429    FRCA violations : Failing to Follow Debt Dispu...
353430    My Father, a XXXX XXXX  acquired an HECM rever...
353431    I have tried to contact cash app about a fraud...
Name: complaint_narrative, Length: 353432, dtype: object

In [8]:
seed = 123
for complaint in complaints.loc[complaints['issue'] == 4, 'complaint_narrative'].sample(3, random_state=seed):
    print(complaint)
    print('-----------------------------')

I just reviewed a copy of my Experian credit report and found the following information to be an error. I am a victim of identity theft and did not make these charges. I reported the theft of my identity to the Federal Trade Commission.
-----------------------------
after my legal separation from my husband he started to open credit in my name with no permission I have a legal case against him but can not find him he has ruined my life his name is XXXX XXXX XXXX   remove I have disputed by mail with all three credit bureaus. act # XXXX owing an alleged {$2200.00} XXXX   XXXX  orig creditor XXXX
-----------------------------
Equifax report a collections account " XXXX XXXX XXXX ''. I have no knowledge of this account. I have not been furnished any proof or verifications of this account. I don't have any signed contract agreements with XXXX XXXX XXXX, verbal agreements nor any paperwork associated with them. A proper investigation according to FCRA was not conducted, word of mouth from t

In [9]:
# remove double x-s
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace(r'[xX][xX]+','', regex=True)

# stripping all numeric values from complaints
complaints['complaint_narrative'] = complaints['complaint_narrative'].str.replace('\d+', '', regex=True)

# removing digit removes "4927" words

In [10]:
#complaints.groupby('issue')['complaint_narrative'].apply(lambda x: x.str.extractall(r'(\w+X+\w+)').groupby(0).size().reset_index(name='count'))

In [11]:
complaints.loc[complaints['complaint_narrative'].str.find('TOXIC') != -1, 'complaint_narrative'].iloc[3]

'  and      , CA  NATIONSTAR/.   #  We need immediate assistance with our   property loan.     is the investor of the property and   is the servicer for the loan. \n\nOur property has a domestic water well and the water is unsafe to drink. The significant and very serious water contaminations in the aquifers that serve our water well include : Lead Hexavalent Chromium ( documented in the movie entitled   ) and Arsenic. These chemicals are listed on the CA Proposition  List of Contaminants that Cause Cancer or Reproductive Harm. Serial testing of the water well by a CA Certified Laboratory defined the contamination to be steady-state as we were informed by the US EPA. There is no filtration system that can reduce these high levels down to potable safe drinking water levels. Our home is now defined* as : HAZARDOUS TOXIC CONTAMINATED UNINHABITABLE POLLUTED NON-POTABLE WATER BLIGHTED AND A NUISANCE ( *as defined by the CA List of Contaminants Proposition  State of CA, EPA Office of Environ

In [12]:
X = complaints[['complaint_narrative']]
y = complaints['issue']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 321, stratify = y)

In [29]:
class LemmaTokenizer:
    def __init__(self):
        self.wnl = WordNetLemmatizer()
    def __call__(self, doc):
        return [self.wnl.lemmatize(t)
                for t in regexp_tokenize(doc, r'(?u)\b\w\w+\b')
                if t.lower() not in stop_words] #word_tokenize(doc)

In [31]:
# note: the default stop words remove 311 words; have to specify it, otherwise it's off
# vect = CountVectorizer(lowercase=True, stop_words='english', ) 

#r”(?u)\b\w\w+\b”
stop_words = set(stopwords.words('english'))
vect = CountVectorizer(tokenizer=LemmaTokenizer())  

X_train_vec = vect.fit_transform(X_train['complaint_narrative'])
X_test_vec = vect.transform(X_test['complaint_narrative'])

In [32]:
vect.vocabulary_

{'filed': 21041,
 'identity': 26025,
 'theft': 52662,
 'report': 44457,
 'learned': 30639,
 'someone': 49205,
 'using': 56769,
 'incarceration': 26968,
 'submitted': 50787,
 'documentation': 16751,
 'transunion': 54306,
 'disregarded': 16377,
 'even': 19356,
 'block': 6908,
 'account': 657,
 'day': 13767,
 'upon': 56567,
 'receiving': 42791,
 'information': 27634,
 'also': 2561,
 'received': 42764,
 'letter': 30898,
 'irs': 29139,
 'stating': 50086,
 'tax': 51959,
 'return': 45535,
 'name': 34672,
 'u': 54969,
 'dept': 14907,
 'educatio': 17714,
 'acct': 784,
 'closed': 9971,
 'send': 47495,
 'written': 59375,
 'notification': 35855,
 'reported': 44470,
 'consumer': 11733,
 'reporting': 44496,
 'agency': 1916,
 'never': 35246,
 'ever': 19380,
 'late': 30427,
 'although': 2607,
 'updated': 56507,
 'chargeoff': 9204,
 'providing': 41521,
 'severe': 47847,
 'scandalous': 46879,
 'action': 1072,
 'destroying': 15195,
 'perfect': 38932,
 'payment': 38572,
 'history': 25087,
 'credit': 12935

In [33]:
len(vect.vocabulary_)

59857

In [34]:
64231 - 59857

4374

### Naive Bayes First

In [35]:
word_counts = pd.DataFrame({
    'words': vect.get_feature_names_out(),
    'frequency': np.array(X_train_vec.sum(axis = 0)).flatten()
})

word_counts.head()

,words,frequency
0,__,24
1,___,48
2,____,44
3,_____,31
4,______,4


In [36]:
word_counts.sort_values('frequency', ascending=False).head(25)

,words,frequency
12935,credit,553340
657,account,509921
44457,report,359288
27634,information,290532
44496,reporting,213314
13933,debt,182786
38572,payment,172136
11733,consumer,168238
10823,company,133872
5542,balance,120282


In [37]:
from sklearn.naive_bayes import MultinomialNB

In [38]:
nb = MultinomialNB().fit(X_train_vec, y_train)

y_pred = nb.predict(X_test_vec)

In [39]:
accuracy_score(y_test, y_pred)

0.8242943479933905

In [40]:
confusion_matrix(y_test, y_pred)

array([[11765,  1971,   441,  3828,   286],
       [  590,  4492,    47,   101,    81],
       [   63,    59,  2816,   113,    36],
       [ 4795,   739,   685, 49529,  1578],
       [   27,    35,    16,    34,  4231]])

In [41]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.68      0.64      0.66     18291
           2       0.62      0.85      0.71      5311
           3       0.70      0.91      0.79      3087
           4       0.92      0.86      0.89     57326
           5       0.68      0.97      0.80      4343

    accuracy                           0.82     88358
   macro avg       0.72      0.85      0.77     88358
weighted avg       0.84      0.82      0.83     88358



### LogReg

In [42]:
logreg = LogisticRegression(max_iter = 10000).fit(X_train_vec, y_train)

y_pred = logreg.predict(X_test_vec)

accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)

array([[11963,   773,   119,  5358,    78],
       [  916,  3990,    29,   342,    34],
       [  152,    28,  2636,   251,    20],
       [ 2551,   125,   105, 54382,   163],
       [  105,    43,    26,   324,  3845]])

In [43]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           1       0.76      0.65      0.70     18291
           2       0.80      0.75      0.78      5311
           3       0.90      0.85      0.88      3087
           4       0.90      0.95      0.92     57326
           5       0.93      0.89      0.91      4343

    accuracy                           0.87     88358
   macro avg       0.86      0.82      0.84     88358
weighted avg       0.87      0.87      0.87     88358



In [26]:
vect = TfidfVectorizer()

X_train_vec = vect.fit_transform(X_train['text'])
X_test_vec = vect.transform(X_test['text'])

KeyError: 'text'

In [ ]:
logreg = LogisticRegression(max_iter = 10000).fit(X_train_vec, y_train)

y_pred = logreg.predict(X_test_vec)

accuracy_score(y_test, y_pred)

confusion_matrix(y_test, y_pred)